In [2]:
pip install ase

     |████████████████████████████████| 2.2 MB 407 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
import os
from subprocess import run, PIPE

from ase.symbols import atomic_numbers

sys.path.insert(0, os.path.abspath("/home/hayden/Programs/mlip-2-mlippy/lib"))
#sys.path.append("/home/hayden/Programs/mlip-2-mlippy/bin")
import mlippy

ModuleNotFoundError: No module named 'ase'

# Use either MLIP or ASE relaxation algorithm

In [ ]:
# ASE
def relax_atom(struct, calculator, steps=100, maxstep=.03, fmax=.05):
    struct.set_calculator(calc=calculator)
    dyn = LBFGS(atoms=struct, maxstep=maxstep)
    dyn.run(fmax=fmax, steps=steps)
    
def relax_atoms(atoms, calculator, steps=100, maxstep=.03, fmax=.05):
    """
    Takes in a list of atoms objects and the calculator to use for EFS calculations
    
    returns a list of relaxed structures, achieved with the LBFGS algorithm
    """

    # Loop through atoms. Their energy, forces, and positions are updated in place
    for struct in atoms:
        relax_atom(struct, calculator=calculator, steps=steps, maxstep=maxstep, fmax=fmax)
  

 MLIP relaxation example
 
results = mlippy.ase_relax(mlip, to_relax, options, relax_options)

# Train a potential

In [ ]:
training_set_path = "train.cfg"
potential_path = "pot_light.mtp"

# Initialize the potential and load in the template
mlippy.initialize()
mlip = mlippy.mtp()
mlip.load_potential(potential_path)

# Load the training set
training_set = mlippy.ase_loadcfgs(training_set_path)

training_options = {"max-iter":"40",
"energy-weight":"1",
"force-weight":"1e-3",
"stress-weight":"1e-4",
"skip-preinit":"TRUE",
"no-mindist-update":"TRUE",
"init-params":"same",
}

In [ ]:
mlippy.ase_train(mlip, training_set, training_options)
errors = mlippy.ase_errors(mlip, training_set)
mlip.save_potential('out/Trained.mtp')

In [ ]:
print("====== ERROR REPORT =======")

with open("out/training_error.txt", 'w') as f:
    for err in errors:
        error_val = errors[err].decode('utf8')
        print(err, '=', error_val, file=f)
        print(err, '=', error_val)

### Calcute the extrapolation grade of the training set

In [ ]:
mlippy.ase_calcgrade('out/Trained.mtp', training_set_path,
                     training_set_path, 'out/tmp.cfg', 
                     state_file="out/state.als")

# Relax a structure

In [ ]:
# potential_path = "out/Trained.mtp"
state_path = 'out/state.als'
mlip = mlippy.mtp(state_path)


# with open("HfNiTi.yaml") as f:
#     yml_vars = yaml.safe_load(f)

# species_map = yml_vars['species']
# for spec in species_map:
# #     element = species_map[spec]
#     mlip.add_atomic_type(atomic_numbers[species_map[spec]])


to_relax = mlippy.ase_loadcfgs('to-relax.cfg')

options = {"select":"TRUE", 
"mtp-filename":state_path,
"load-state":state_path,
"save-selected":"out/selected_py.cfg",
"threshold":"2",
"threshold-break":"5",
"abinitio":"null"
}

relax_options = {"iteration-limit":"990",
                "min-dist":"1.5"}

We will now use our potential to relax a list of ASE atoms. If these structures do not relax properly they will have an energy value of `None`

In [ ]:
results = mlippy.ase_relax(mlip, to_relax, options, relax_options)

In [ ]:
relaxed = []
unrelaxed = []

for cfg in results:
    if (cfg.energy != None):
        relaxed.append(cfg)
    else:
        unrelaxed.append(cfg)
        
mlippy.ase_savecfgs('out/relaxed.cfg', relaxed)
mlippy.ase_savecfgs('out/unrelaxed.cfg', unrelaxed)

In [ ]:
unrelaxed

# Selecting new training structures

In [ ]:
mlip = mlippy.mtp('out/Trained.mtp')

select_options = {"selection-limit":"20",
                 "als-filename":"out/state.als"}

diff_py = mlippy.ase_select(mlip, results, training_set, select_options)

In [ ]:
# class newMlip(mlippy.mtp):
def ase_calc_grade():
    print("hello there")

In [ ]:
mlp = newMlip

In [ ]:
type(mlippy)

In [ ]:
mlippy.ase_calcgrade = ase_calc_grade

In [ ]:
mlippy.ase_calcgrade()